In [145]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import numba
from numba import jit

In [11]:
import ar1_approx as ar

# Exercise 1

In [28]:
#parameters:
alpha_k=0.297
alpha_l=0.650
delta = 0.154
psi = 1.08
w = 0.7
r= 0.04
z = 1
mu=0
rho=0.7605
sigma_z=0.213
sizez=9
betafirm = (1 / (1 + r))
sigma_eps=sigma_z*((1 - rho ** 2) ** (1 / 2))

In [177]:
dens = 5
kstar = (((alpha_k/(r+delta))**(1-alpha_l))*((alpha_l/w)**(alpha_l)))**(1-alpha_k-alpha_l)      
kbar = 2*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
kvec = K[::-1]
sizek = kvec.shape[0]
z_grid, pi=ar.addacooper(sizez,mu,rho,sigma_z)
realz=np.exp(z_grid)
kvec[5]*(1-delta)-kvec[0]

1.0842021724855044e-19

In [158]:
# operating profits, op
op = ((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l)))*
      ((kvec ** alpha_k) ** (1 / (1 - alpha_l))))
#define profit as a function of both z and k:
profit=np.dot(np.array([op]).T,np.array([realz**(1/(1-alpha_l))])).T
# firm cash flow, e:
e = np.zeros((sizez,sizek, sizek))
for l in range(sizez):
    for i in range(sizek):
        for j in range(sizek):
                e[l,i, j] = (profit[l,i] - kvec[j] + ((1 - delta) * kvec[i]) -
                           ((psi / 2) * ((kvec[j] - ((1 - delta) * kvec[i])) ** 2)
                            / kvec[i]))


In [159]:
VFtol = 1e-5
VFdist = 7.0
VFmaxiter = 200
V = np.zeros((sizez,sizek))  # initial guess at value function
Vmat = np.zeros((sizez, sizek, sizek))  # initialize Vmat matrix
Vstore = np.zeros((sizez,sizek, VFmaxiter))  # initialize Vstore array
VFiter = 1

In [212]:
@jit
def VFI(VFtol, VFmaxiter):
    VFtol = 1e-5
    VFdist = 7.0
    VFmaxiter = 300
    V = np.zeros((sizez,sizek))  # initial guess at value function
    Vmat = np.zeros((sizez, sizek, sizek))  # initialize Vmat matrix
    VFiter = 1
    PF=0
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = V
        for h in range(sizez):
            for i in range(sizek):  # loop over k
                for j in range(sizek):  # loop over k'
                    Vmat[h,i,j] = e[h,i,j] + betafirm * np.dot(pi[h,:],V[:,j])
        V = Vmat.max(axis=2)  # apply max operator to Vmat (to get V(z,k))
        PF = np.argmax(Vmat, axis=2)  # find the index of the optimal k'
        VFdist = (np.absolute(V - TV)).max()  # check distance between value
        # function for this iteration and value function from past iteration
        VFiter += 1
    return V, PF




In [182]:
V, PF=VFI(VFtol, VFmaxiter)
optK = kvec[PF]
optI = optK - (1 - delta) * kvec


# Exercise 2

In [219]:
psi1=0.03
f = np.zeros((sizez,sizek, sizek))
g = np.zeros((sizez,sizek))
for l in range(sizez):
    for i in range(sizek):
        for j in range(sizek):
                f[l,i, j] = (profit[l,i] - kvec[j] + ((1 - delta) * kvec[i]) - psi1*kvec[i])
for l in range(sizez):
    for i in range(sizek):
            g[l,i] = (profit[l,i] - kvec[j] + ((1 - delta) * kvec[i]))    

In [236]:
@jit
def VFInew(VFtol, VFmaxiter):
    VFtol = 1e-5
    VFdist = 7.0
    VFmaxiter = 300
    V = np.zeros((sizez,sizek))  # initial guess at value function
    Vmat1 = np.zeros((sizez, sizek, sizek))  # initialize Vmat matrix
    V2 = np.zeros((sizez, sizek)) 
    VFiter = 1
    PF=np.copy(V)
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = V
        for h in range(sizez):
            for i in range(sizek):
                s=max((i-dens),0)
                V2[h,i] = g[h,i] + betafirm * np.dot(pi[h,:],V[:,s])
                for j in range(sizek):  # loop over k'
                    Vmat1[h,i,j] = f[h,i,j] + betafirm * np.dot(pi[h,:],V[:,j])     
        V1=Vmat1.max(axis=2)  # apply max operator to Vmat (to get V(z,k))
        for h in range(sizez):
            for i in range(sizek):
                V[h,i]=max(V1[h,i],V2[h,i])
                if V[h,i]==V1[h,i]:
                    PF[h,i]=np.argmax(Vmat1,axis=2)[h,i]
                else:
                    PF[h,i]=max((i-dens),0)
        VFdist = (np.absolute(V - TV)).max()  # check distance between value
        # function for this iteration and value function from past iteration
        VFiter += 1
    return V, PF


In [237]:
VFInew(VFtol, VFmaxiter)


(array([[ -6.52109858e-06,   2.46317948e-05,   5.68190277e-05, ...,
           1.63913600e+00,   1.69436878e+00,   1.75146432e+00],
        [  1.58087512e-04,   1.93979313e-04,   2.31041882e-04, ...,
           1.74253580e+00,   1.80074535e+00,   1.86090336e+00],
        [  3.01907632e-04,   3.41939862e-04,   3.83262058e-04, ...,
           1.83287719e+00,   1.89368758e+00,   1.95652129e+00],
        ..., 
        [  1.27106414e-03,   1.33899736e-03,   1.40902378e-03, ...,
           2.44165814e+00,   2.51999469e+00,   2.60085912e+00],
        [  1.89620294e-03,   1.98213324e-03,   2.07067487e-03, ...,
           2.83434250e+00,   2.92398401e+00,   3.01647886e+00],
        [  3.92564004e-03,   4.06999569e-03,   4.21864466e-03, ...,
           4.10914449e+00,   4.23548620e+00,   4.36573782e+00]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]

In [226]:
VFtol = 1e-5
VFdist = 7.0
VFmaxiter = 100
V = np.zeros((sizez,sizek))  # initial guess at value function
Vmat1 = np.zeros((sizez, sizek, sizek))  # initialize Vmat matrix
V2 = np.zeros((sizez, sizek)) 
VFiter = 1
PF=np.copy(V)
while VFdist > VFtol and VFiter < VFmaxiter:
    TV = V
    for h in range(sizez):
        for i in range(sizek):
            s=max((i-dens),0)
            V2[h,i] = g[h,i] + betafirm * np.dot(pi[h,:],V[:,s])
            for j in range(sizek):  # loop over k'
                Vmat1[h,i,j] = f[h,i,j] + betafirm * np.dot(pi[h,:],V[:,j])     
    V1=Vmat1.max(axis=2)  # apply max operator to Vmat (to get V(z,k))
    for h in range(sizez):
        for i in range(sizek):
            V[h,i]=max(V1[h,i],V2[h,i])
            if V[h,i]==V1[h,i]:
                PF[h,i]=np.argmax(Vmat1,axis=2)[h,i]
            else:
                PF[h,i]=max((i-dens),0)
    VFdist = (np.absolute(V - TV)).max()  # check distance between value
        # function for this iteration and value function from past iteration
    VFiter += 1

TypeError: 'int' object does not support item assignment